In [1]:
import pandas as pd
from itertools import combinations
import itertools


import networkx as nx

## Data Load

In [2]:
p1_ipc = pd.read_excel("./data/1.wearables_2011-2013_original_part.xlsx", sheet_name="ipc_info")
p1_td = pd.read_excel("./data/1.wearables_2011-2013_original_part.xlsx", sheet_name="transaction_data")

p2_ipc = pd.read_excel("./data/2.wearables_2014-2016_original_part.xlsx", sheet_name="ipc_info")
p2_td = pd.read_excel("./data/2.wearables_2014-2016_original_part.xlsx", sheet_name="transaction_data")

p3_ipc = pd.read_excel("./data/3.wearables_2017-2019_original_part.xlsx", sheet_name="ipc_info")
p3_td = pd.read_excel("./data/3.wearables_2017-2019_original_part.xlsx", sheet_name="transaction_data")

p1_td

,p_id,ori_ipc,ipc_info
0,4,A61N,37 Medical equipment
1,4,H01Q,"35 Signal transmission, telecommunications"
2,5,G05D,-
3,5,A61L,37 Medical equipment
4,5,B05B,-
...,...,...,...
872,655,H04B,"35 Signal transmission, telecommunications"
873,655,H03B,"35 Signal transmission, telecommunications"
874,655,A61N,37 Medical equipment
875,656,A61B,37 Medical equipment


In [3]:
p1_ipc.head()

,ipc,tech_group
0,G09B,"35 Signal transmission, telecommunications"
1,G09C,"35 Signal transmission, telecommunications"
2,H01P,"35 Signal transmission, telecommunications"
3,H01Q,"35 Signal transmission, telecommunications"
4,H01S,"35 Signal transmission, telecommunications"


## 가능한 모든 Combination 만들기

In [4]:
p1_df_35 = p1_ipc["ipc"].loc[p1_ipc["tech_group"].str.startswith("35")]
p1_df_37 = p1_ipc["ipc"].loc[p1_ipc["tech_group"].str.startswith("37")]
p1_combi_df = pd.DataFrame(list(itertools.product(p1_df_35, p1_df_37)), columns=["35", "37"])

p2_df_35 = p2_ipc["ipc"].loc[p2_ipc["tech_group"].str.startswith("35")]
p2_df_37 = p2_ipc["ipc"].loc[p2_ipc["tech_group"].str.startswith("37")]
p2_combi_df = pd.DataFrame(list(itertools.product(p2_df_35, p2_df_37)), columns=["35", "37"])

p3_df_35 = p3_ipc["ipc"].loc[p3_ipc["tech_group"].str.startswith("35")]
p3_df_37 = p3_ipc["ipc"].loc[p3_ipc["tech_group"].str.startswith("37")]
p3_combi_df = pd.DataFrame(list(itertools.product(p3_df_35, p3_df_37)), columns=["35", "37"])

print(p1_combi_df.shape, p2_combi_df.shape, p3_combi_df.shape)
p1_combi_df.head(15)


(378, 2) (378, 2) (378, 2)


,35,37
0,G09B,A61B
1,G09B,A61C
2,G09B,A61D
3,G09B,A61F
4,G09B,A61G
5,G09B,A61H
6,G09B,A61J
7,G09B,A61L
8,G09B,A61M
9,G09B,A61N


## transaction_data 에서 35/37/none ipc 추출

In [5]:
p1_td_35 = p1_td.set_index(["p_id"])["ori_ipc"].loc[p1_td.set_index(["p_id"])["ipc_info"].str.startswith("35")].reset_index()
p1_td_37 = p1_td.set_index(["p_id"])["ori_ipc"].loc[p1_td.set_index(["p_id"])["ipc_info"].str.startswith("37")].reset_index()
p1_td_none = p1_td.set_index(["p_id"])["ori_ipc"].loc[p1_td.set_index(["p_id"])["ipc_info"].str.startswith("-")].reset_index()

p1_td_df = pd.merge(p1_td_35, p1_td_37, on=["p_id"], how="outer").groupby(["p_id"]).apply(lambda x: x[:])
p1_td_df = pd.merge(p1_td_df, p1_td_none, on=["p_id"], how="outer").groupby(["p_id"]).apply(lambda x: x[:]).fillna("")
p1_td_df.columns = ["p_id", "35", "37", "none"]


p2_td_35 = p2_td.set_index(["p_id"])["ori_ipc"].loc[p2_td.set_index(["p_id"])["ipc_info"].str.startswith("35")].reset_index()
p2_td_37 = p2_td.set_index(["p_id"])["ori_ipc"].loc[p2_td.set_index(["p_id"])["ipc_info"].str.startswith("37")].reset_index()
p2_td_none = p2_td.set_index(["p_id"])["ori_ipc"].loc[p2_td.set_index(["p_id"])["ipc_info"].str.startswith("-")].reset_index()

p2_td_df = pd.merge(p2_td_35, p2_td_37, on=["p_id"], how="outer").groupby(["p_id"]).apply(lambda x: x[:])
p2_td_df = pd.merge(p2_td_df, p2_td_none, on=["p_id"], how="outer").groupby(["p_id"]).apply(lambda x: x[:]).fillna("")
p2_td_df.columns = ["p_id", "35", "37", "none"]


p3_td_35 = p3_td.set_index(["p_id"])["ori_ipc"].loc[p3_td.set_index(["p_id"])["ipc_info"].str.startswith("35")].reset_index()
p3_td_37 = p3_td.set_index(["p_id"])["ori_ipc"].loc[p3_td.set_index(["p_id"])["ipc_info"].str.startswith("37")].reset_index()
p3_td_none = p3_td.set_index(["p_id"])["ori_ipc"].loc[p3_td.set_index(["p_id"])["ipc_info"].str.startswith("-")].reset_index()

p3_td_df = pd.merge(p3_td_35, p3_td_37, on=["p_id"], how="outer").groupby(["p_id"]).apply(lambda x: x[:])
p3_td_df = pd.merge(p3_td_df, p3_td_none, on=["p_id"], how="outer").groupby(["p_id"]).apply(lambda x: x[:]).fillna("")
p3_td_df.columns = ["p_id", "35", "37", "none"]


print(p1_td_df.shape, p2_td_df.shape, p3_td_df.shape)
p1_td_df.head(15)

(548, 4) (31595, 4) (15507, 4)


,p_id,35,37,none
0,4,H01Q,A61N,
1,5,H03F,A61L,G05D
2,5,H03F,A61L,B05B
3,5,H03F,A61L,G06F
4,6,H03G,A61F,
5,8,G09B,A61B,G06Q
6,9,H04L,A61B,G06Q
7,9,H04L,A61B,G06F
8,10,G09B,A61B,G06K
9,10,G09B,A61B,H04N


## Pairs not Exist in Period

In [6]:
# 비교하기 위해 ipc concat

p1_combi_df["combi"] = p1_combi_df["35"]+p1_combi_df["37"]
p1_td_df["combi"] = p1_td_df["35"]+p1_td_df["37"]+p1_td_df["none"]
p1_td_df["combi_3537"] = p1_td_df["35"]+p1_td_df["37"]

p2_combi_df["combi"] = p2_combi_df["35"]+p2_combi_df["37"]
p2_td_df["combi"] = p2_td_df["35"]+p2_td_df["37"]+p2_td_df["none"]
p2_td_df["combi_3537"] = p2_td_df["35"]+p2_td_df["37"]

p3_combi_df["combi"] = p3_combi_df["35"]+p3_combi_df["37"]
p3_td_df["combi"] = p3_td_df["35"]+p3_td_df["37"]+p3_td_df["none"]
p3_td_df["combi_3537"] = p3_td_df["35"]+p3_td_df["37"]

print(p1_combi_df["combi"].head(), "\n", p1_td_df["combi"].head(), p1_combi_df.shape, p1_td_df.shape)
# print(p2_combi_df["combi"].head(), p2_td_df["combi"].head())
# print(p3_combi_df["combi"].head(),"\n", p3_td_df["combi"].head())

0    G09BA61B
1    G09BA61C
2    G09BA61D
3    G09BA61F
4    G09BA61G
Name: combi, dtype: object 
 0        H01QA61N
1    H03FA61LG05D
2    H03FA61LB05B
3    H03FA61LG06F
4        H03GA61F
Name: combi, dtype: object (378, 3) (548, 6)


In [7]:
p1_exist_pair = list(set(p1_td_df["combi_3537"].unique()))
p2_exist_pair = list(set(p2_td_df["combi_3537"].unique()))
p3_exist_pair = list(set(p3_td_df["combi_3537"].unique()))

train_df = p1_combi_df[p1_combi_df["combi"].isin(p1_exist_pair)==False]
test_df = p2_combi_df[p2_combi_df["combi"].isin(p2_exist_pair)==False]

print(f"pairs in period 1: {len(p1_exist_pair)}\tpairs not in period 1: {len(train_df)}")
print(f"pairs in period 2: {len(p2_exist_pair)}\tpairs not in period 2: {len(test_df)}")

pairs in period 1: 71	pairs not in period 1: 307
pairs in period 2: 165	pairs not in period 2: 213


## Y label - Pairs Exist / not Exist in next period

In [8]:
train_df["y"] = 0
test_df["y"] = 0

period_2_idx = list(train_df[train_df["combi"].isin(p2_exist_pair)==True].index)
period_3_idx = list(test_df[test_df["combi"].isin(p3_exist_pair)==True].index)

train_df.loc[period_2_idx, "y"] = 1
test_df.loc[period_3_idx, "y"] = 1 

print(train_df["y"].value_counts(), "\n", test_df["y"].value_counts())

0    208
1     99
Name: y, dtype: int64 
 0    168
1     45
Name: y, dtype: int64


/tmp/ipykernel_72899/3215633791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["y"] = 0
/tmp/ipykernel_72899/3215633791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["y"] = 0


## Common Neighbor

In [9]:
train_df.merge(p1_td_df)

,35,37,combi,y,p_id,none,combi_3537


In [14]:
p1_td.groupby(["p_id"])["ori_ipc"].apply(list)

p_id
4                        [A61N, H01Q]
5      [G05D, A61L, B05B, G06F, H03F]
6                        [A61F, H03G]
8                  [G09B, A61B, G06Q]
9            [A61B, G06Q, G06F, H04L]
                    ...              
642          [H02J, F21L, A61B, H01M]
651                      [H01Q, A61B]
654          [H01S, H01L, A61N, G21G]
655          [H04R, H04B, H03B, A61N]
656                      [A61B, H04B]
Name: ori_ipc, Length: 252, dtype: object

In [10]:
# p1_td.groupby(["p_id"])["ori_ipc"].apply(list).apply(len)-1
cn = p1_td.groupby(["p_id"])["ori_ipc"].apply(list).apply(len)-1
cn.columns = ["p_id", "ori_ipc"]
cn

p_id
4      1
5      4
6      1
8      2
9      3
      ..
642    3
651    1
654    3
655    3
656    1
Name: ori_ipc, Length: 252, dtype: int64

In [11]:
p1_td.merge(cn, on="p_id")

,p_id,ori_ipc_x,ipc_info,ori_ipc_y
0,4,A61N,37 Medical equipment,1
1,4,H01Q,"35 Signal transmission, telecommunications",1
2,5,G05D,-,4
3,5,A61L,37 Medical equipment,4
4,5,B05B,-,4
...,...,...,...,...
872,655,H04B,"35 Signal transmission, telecommunications",3
873,655,H03B,"35 Signal transmission, telecommunications",3
874,655,A61N,37 Medical equipment,3
875,656,A61B,37 Medical equipment,1


In [12]:
p1_td.groupby(["p_id"])["ori_ipc"].apply(list).apply(len)

p_id
4      2
5      5
6      2
8      3
9      4
      ..
642    4
651    2
654    4
655    4
656    2
Name: ori_ipc, Length: 252, dtype: int64

In [13]:
p1_td

,p_id,ori_ipc,ipc_info
0,4,A61N,37 Medical equipment
1,4,H01Q,"35 Signal transmission, telecommunications"
2,5,G05D,-
3,5,A61L,37 Medical equipment
4,5,B05B,-
...,...,...,...
872,655,H04B,"35 Signal transmission, telecommunications"
873,655,H03B,"35 Signal transmission, telecommunications"
874,655,A61N,37 Medical equipment
875,656,A61B,37 Medical equipment
